In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Glucocorticoid_Sensitivity/GSE57795'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"in vivo dexamethasone-induced gene expression in pediatric acute lymphoblastic leukemia patient-derived xenografts"
!Series_summary	"Glucocorticoids are critical components of combination chemotherapy regimens in pediatric acute lymphoblastic leukemia (ALL). The pro-apoptotic BIM protein is an important mediator of glucocorticoid-induced apoptosis in normal and malignant lymphocytes, while the anti-apoptotic BCL2 confers resistance. The signaling pathways regulating BIM and BCL2 expression in glucocorticoid-treated lymphoid cells remain unclear. In this study, pediatric ALL patient-derived xenografts (PDXs) inherently sensitive or resistant to glucocorticoids were exposed to dexamethasone in vivo. In order to understand the basis for differential in vivo glucocorticoid sensitivity of PDXs, microarray analysis of gene expression was carried out on 5 each of dexamethasone-sensitive and resistant PDXs . This provided a global understanding of dexamet

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Define sample_characteristics based on STEP 1
sample_characteristics = {
    0: ['strain: NOD/SCID'], 
    1: ['injection: ALL patient-derived xenograft cells were inoculated by tail-vein injection'], 
    2: ['age (mouse): xenograft cells injected at 6-10 weeks'], 
    3: ['treatment: control', 'treatment: 8h dexamethasone'], 
    4: ['tissue: xenograft cells (>95% hCD45+ cells) from mouse spleens'], 
    5: ['dexamethasone response: Sensitive patient', 'dexamethasone response: Resistant patient']
}

# Check gene expression data availability
is_gene_available = True  # Based on the description, microarray gene expression data is available

# Check for variable availability in the sample characteristics
for key, value in sample_characteristics.items():
    if any("dexamethasone response" in v for v in value):
        trait_row = key
    elif any("age (mouse)" in v for v in value):
        age_row = key
    elif any("gender" in v for v in value):
        gender_row = key

# Function for data type conversion
def extract_value(cell):
    return cell.split(': ')[1] if ': ' in cell else None

def convert_trait(value):
    value = extract_value(value)
    if value == "Sensitive patient":
        return 1
    elif value == "Resistant patient":
        return 0
    return None

def convert_age(value):
    value = extract_value(value)
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    value = extract_value(value)
    if 'male' in value.lower():
        return 1
    elif 'female' in value.lower():
        return 0
    return None

# Save cohort information
save_cohort_info('GSE57795', './preprocessed/Glucocorticoid_Sensitivity/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction, if applicable
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Glucocorticoid_Sensitivity', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Glucocorticoid_Sensitivity/trait_data/GSE57795.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1388640': [1, None], 'GSM1388641': [1, None], 'GSM1388642': [1, None], 'GSM1388643': [1, None], 'GSM1388644': [1, None], 'GSM1388645': [0, None], 'GSM1388646': [0, None], 'GSM1388647': [0, None], 'GSM1388648': [0, None], 'GSM1388649': [0, None], 'GSM1388650': [0, None], 'GSM1388651': [0, None], 'GSM1388652': [0, None], 'GSM1388653': [0, None], 'GSM1388654': [0, None], 'GSM1388655': [0, None], 'GSM1388656': [0, None], 'GSM1388657': [1, None], 'GSM1388658': [1, None], 'GSM1388659': [1, None], 'GSM1388660': [1, None], 'GSM1388661': [1, None], 'GSM1388662': [1, None], 'GSM1388663': [1, None], 'GSM1388664': [1, None], 'GSM1388665': [1, None], 'GSM1388666': [1, None], 'GSM1388667': [1, None], 'GSM1388668': [1, None], 'GSM1388669': [1, None], 'GSM1388670': [1, None], 'GSM1388671': [1, None], 'GSM1388672': [1, None], 'GSM1388673': [1, None], 'GSM1388674': [1, None], 'GSM1388675': [1, None], 'GSM1388676': [1, None], 'GSM1388677': [1, None], 'GSM1388678': [1, None], 'GSM1388679': [1, None],

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1825594', 'ILMN_1810803', 'ILMN_1722532', 'ILMN_1884413', 'ILMN_1906034'], 'Species': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Source': ['Unigene', 'RefSeq', 'RefSeq', 'Unigene', 'Unigene'], 'Search_Key': ['ILMN_89282', 'ILMN_35826', 'ILMN_25544', 'ILMN_132331', 'ILMN_105017'], 'Transcript': ['ILMN_89282', 'ILMN_35826', 'ILMN_25544', 'ILMN_132331', 'ILMN_105017'], 'ILMN_Gene': ['HS.388528', 'LOC441782', 'JMJD1A', 'HS.580150', 'HS.540210'], 'Source_Reference_ID': ['Hs.388528', 'XM_497527.2', 'NM_018433.3', 'Hs.580150', 'Hs.540210'], 'RefSeq_ID': [nan, 'XM_497527.2', 'NM_018433.3', nan, nan], 'Unigene_ID': ['Hs.388528', nan, nan, 'Hs.580150', 'Hs.540210'], 'Entrez_Gene_ID': [nan, 441782.0, 55818.0, nan, nan], 'GI': [23525203.0, 89042416.0, 46358420.0, 7376124.0, 5437312.0], 'Accession': ['BU678343', 'XM_497527.2', 'NM_018433.3', 'AW629334', 'AI818233'], 'Symbol': [nan, 'LOC441782', 'JMJD1A', nan, nan], 'Prot

### Step 5 has revealed a preview of the gene annotation: Unknown Step

In [7]:
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 1. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 2. Apply the mapping and obtain the resulting gene expression dataframe
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Glucocorticoid_Sensitivity/gene_data/GSE57795.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Glucocorticoid_Sensitivity')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE57795', './preprocessed/Glucocorticoid_Sensitivity/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Glucocorticoid_Sensitivity/GSE57795.csv'
    unbiased_merged_data.to_csv(csv_path)


Quartiles for 'Glucocorticoid_Sensitivity':
  25%: nan
  50% (Median): nan
  75%: nan
Min: nan
Max: nan
The distribution of the feature 'Glucocorticoid_Sensitivity' in this dataset is fine.

Quartiles for 'Age':
  25%: nan
  50% (Median): nan
  75%: nan
Min: nan
Max: nan
The distribution of the feature 'Age' in this dataset is fine.

